In [1]:
import argparse
import glob
import pathlib
import sqlite3
import sys

import lancedb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from cytotable import convert, presets
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from pycytominer import annotate, feature_select, normalize
from pycytominer.cyto_utils import output

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

/home/lippincm/miniforge3/envs/cellprofiler_timelapse_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set paths and variables

In [2]:
sqlite_path = pathlib.Path(
    "../../4.cellprofiler_analysis/analysis_output/endpoint_whole_image"
).resolve()
# get the files in the children directories
sqlite_files = glob.glob(f"{sqlite_path}/**/*.sqlite", recursive=True)

In [3]:
# load in platemap file as a pandas dataframe
platemap_path = pathlib.Path("../../data/platemap_AnnexinV_2ch.csv").resolve()
platemap_df = pd.read_csv(platemap_path)

# directory where the annotated parquet files are saved to
output_dir = pathlib.Path("../data/endpoint_whole_image/").resolve()
output_dir.mkdir(exist_ok=True, parents=True)

normalized_data_dir = pathlib.Path(
    output_dir, "normalized_whole_image.parquet"
).resolve()
feature_selected_data_dir = pathlib.Path(
    output_dir, "feature_selected_whole_image.parquet"
).resolve()

## Convert

In [4]:
preset = """SELECT * FROM Per_Image;"""

In [5]:
blacklist_keywords = [
    "Skeleton",
    "URL",
    "ExecutionTime",
    "Frame",
    "Group",
    "Height",
    "Width",
    "MD5",
    "Scaling",
    "Series",
]

In [6]:
list_of_dfs = []
for file in sqlite_files:
    source_path = pathlib.Path(file)
    output_file_dir = output_dir / source_path.stem
    # get the path to the sqlite file
    with sqlite3.connect(source_path) as conn:
        query = "SELECT * FROM Per_Image;"
        df = pd.read_sql_query(query, conn)
    list_of_dfs.append(df)

df = pd.concat(list_of_dfs, ignore_index=True)
df = df.drop_duplicates()
# Save the DataFrame to a Parquet file

# df.to_parquet(output_parquet_path, index=False)
list_of_col_to_remove = []
for col in df.columns:
    for keyword in blacklist_keywords:
        if keyword in col:
            list_of_col_to_remove.append(col)
df.drop(columns=list_of_col_to_remove, inplace=True)

for col in df.columns:
    if col.startswith("Image_"):
        df.rename(columns={col: col.replace("Image_", "")}, inplace=True)

## Annotate

In [7]:
# add metadata from platemap file to extracted single cell features
annotated_df = annotate(
    profiles=df,
    platemap=platemap_df,
    join_on=["Metadata_well", "Metadata_Well"],
)
# drop duplicate columns
annotated_df.drop_duplicates(inplace=True)
columns_to_drop = [
    "ImageNumber",
    "FileName_AnnexinV",
    "FileName_DNA",
    "PathName_AnnexinV",
    "PathName_DNA",
]
annotated_df.drop(columns=columns_to_drop, inplace=True)
annotated_df.head()

,Metadata_plate,Metadata_compound,Metadata_dose,Metadata_control,Metadata_Channel,Metadata_FOV,Metadata_FileLocation,Metadata_Time,Metadata_Well,Metadata_Z_slice,...,Texture_SumVariance_DNA_3_02_256,Texture_SumVariance_DNA_3_03_256,Texture_Variance_AnnexinV_3_00_256,Texture_Variance_AnnexinV_3_01_256,Texture_Variance_AnnexinV_3_02_256,Texture_Variance_AnnexinV_3_03_256,Texture_Variance_DNA_3_00_256,Texture_Variance_DNA_3_01_256,Texture_Variance_DNA_3_02_256,Texture_Variance_DNA_3_03_256
0,1,Staurosporine,1.22,test,None,0004,None,0014,C-04,0001,...,0.204565,0.197763,0.254029,0.254100,0.253996,0.254100,0.058763,0.058764,0.058709,0.058764
1,1,Staurosporine,1.22,test,None,0002,None,0014,C-04,0001,...,0.208650,0.200193,0.218253,0.218183,0.218197,0.218182,0.060559,0.060571,0.060572,0.060570
2,1,Staurosporine,1.22,test,None,0003,None,0014,C-04,0001,...,0.088057,0.082560,0.284062,0.284278,0.284222,0.284278,0.025187,0.025211,0.025692,0.025212
3,1,Staurosporine,1.22,test,None,0001,None,0014,C-04,0001,...,0.183499,0.176284,0.245752,0.245460,0.245324,0.245460,0.053496,0.053502,0.053459,0.053502
48,1,Staurosporine,78.13,test,None,0004,None,0014,D-10,0001,...,0.576456,0.557999,0.380764,0.380807,0.380852,0.380806,0.152995,0.153092,0.152966,0.153092


## Normalize

In [8]:
metadata_columns = [x for x in annotated_df.columns if "Metadata_" in x]
feature_columns = [x for x in annotated_df.columns if "Metadata_" not in x]

In [9]:
normalized_df = normalize(
    # df with annotated raw merged single cell features
    profiles=annotated_df,
    # specify samples used as normalization reference (negative control)
    samples="Metadata_compound == 'Staurosporine' and Metadata_dose == 0.0",
    # normalization method used
    method="standardize",
    features=feature_columns,
    meta_features=metadata_columns,
)

## Feature selection

In [10]:
# define operations to be performed on the data
# list of operations for feature select function to use on input profile
feature_select_ops = [
    "variance_threshold",
    "blocklist",
    "drop_na_columns",
    "correlation_threshold",
]

feature_select_df = feature_select(
    normalized_df,
    operation=feature_select_ops,
    # specify features to be used for feature selection
    features=feature_columns,
)


print(f"Number of features before feature selection: {normalized_df.shape[1]}")
print(f"Number of features after feature selection: {feature_select_df.shape[1]}")
feature_select_df.head()

Number of features before feature selection: 136
Number of features after feature selection: 31


,Metadata_plate,Metadata_compound,Metadata_dose,Metadata_control,Metadata_Channel,Metadata_FOV,Metadata_FileLocation,Metadata_Time,Metadata_Well,Metadata_Z_slice,...,Texture_DifferenceEntropy_AnnexinV_3_00_256,Texture_DifferenceEntropy_DNA_3_00_256,Texture_DifferenceVariance_AnnexinV_3_03_256,Texture_DifferenceVariance_DNA_3_00_256,Texture_InfoMeas1_AnnexinV_3_00_256,Texture_InfoMeas1_DNA_3_03_256,Texture_InfoMeas2_AnnexinV_3_03_256,Texture_InverseDifferenceMoment_AnnexinV_3_00_256,Texture_SumVariance_DNA_3_03_256,Texture_Variance_AnnexinV_3_02_256
0,1,Staurosporine,1.22,test,None,0004,None,0014,C-04,0001,...,-1.110862,0.606494,0.884984,-0.791035,-2.241106,-0.819993,0.473630,1.205834,1.411462,-1.206193
1,1,Staurosporine,1.22,test,None,0002,None,0014,C-04,0001,...,-2.003338,1.068383,1.500758,-1.526473,-1.202551,1.091309,-0.650447,1.964588,1.485453,-2.165498
2,1,Staurosporine,1.22,test,None,0003,None,0014,C-04,0001,...,-0.601772,-1.718307,0.899249,-1.245227,-1.165456,0.598269,0.267862,0.873253,-2.096121,-0.396190
3,1,Staurosporine,1.22,test,None,0001,None,0014,C-04,0001,...,-1.984114,0.603107,0.349924,-0.790168,-2.994923,1.594837,0.747283,2.022597,0.757480,-1.438568
48,1,Staurosporine,78.13,test,None,0004,None,0014,D-10,0001,...,4.430621,0.418727,-0.579661,-0.731447,-2.363835,-7.952185,3.768172,-4.995891,12.379506,2.193259
